<a href="https://colab.research.google.com/github/JohnW-07/IRON_SLAM/blob/Shaheen/Copy_of_RVEColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# File Settings
INPUT_FILE = "trimmed construction.mp4"

# this is the output file that will me exported to a "RVE" folder under the "My Drive" folder in Google Drive. WILL OVERWRITE ANY EXISTING FILES
OUTPUT_FILE = "output2.mkv" # mkv is recommended, as it supports more codecs {mkv, mp4, avi, mov}

KILL_RUNTIME_AFTER_RENDER = True # Terminates the runtime to save on your colab compute. If copying to drive fails, manual intervention is required to download files.
# Interpolate Settings

# https://github.com/TNTwise/real-video-enhancer-models/releases/tag/models/ is where all model files for RVE are stored
# you can download them from there, or try custom models from https://openmodeldb.info/
INTERPOLATE = True # {True, False}
INTERPOLATE_FACTOR = 2
SCENE_DETECT_METHOD = "pyscenedetect" # {pyscenedetect, none}
SCENE_DETECT_SENSATIVITY = "3.5" # {0 - 9.9} lower is more sensative

"""
Internal Interpolate Models:
rife4.6.pkl - Speed
rife4.7.pkl - Smoothness
rife4.18.pkl - IRL
rife4.22.pkl - Animation
rife4.25.pkl - General
GMFSS.pkl - Animation
GMFSS_PRO.pkl - Animation
GIMMVFI_RAFT.pth - IRL
"""
INTERPOLATE_MODEL = "rife4.25.pkl" # {Internal model, (link to model path)}


# Upscale Settings
UPSCALE = True # {True, False}
"""
4xNomos8k_span_otf_weak.pth - Realistic 4x High Quality Input
4xNomos8k_span_otf_medium.pth - Realistic 4x Medium Quality Input
4xNomos8k_span_otf_strong.pth - Realistic 4x Low Quality Input
2x_BHI_SpanPlusDynamic_Light.pth - Realistic 2x High Quality Input (pytorch only, experimental)

2x_ModernSpanimationV2.pth - Animation 2x
2x_ModernSpanimationV3.pth - Animation 2x (pytorch only, experimental)
2x_AnimeJaNai_V2_Compact_36k.pth - Animation 2x
2x_AnimeJaNai_HD_V3_Sharp1_Compact_430k.pth - Animation 2x
up2x-latest-conservative.pth - Animation 2x (pytorch only, slow)
up3x-latest-conservative.pth - Animation 3x (pytorch only, slow)
up4x-latest-conservative.pth - Animation 4x (pytorch only, slow)
"""
UPSCALE_MODEL = "2x_ModernSpanimationV2.pth" # {(link to model path)}

# FFMpeg Settings
VIDEO_ENCODER = "libx264" # {libx264,libx265,vp9,av1,prores,ffv1,x264_vulkan,x264_nvenc,x265_nvenc}
AUDIO_ENCODER = "copy_audio" # {aac,libmp3lame,opus,copy_audio}
SUBTITLE_ENCODER = "copy_subtitle" # {srt,ass,webvtt,copy_subtitle}

# Backend Settings
BACKEND = "pytorch" # {pytorch, tensorrt} TensorRT is fastest, but takes a long time to install deps.


In [ ]:
# will move output file to drive when done for permident storage
import os
import subprocess
import shutil
from google.colab import drive, output

# mount drive and move output there for permadent storage

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!rm -rf real-video-enhancer/
!git clone https://github.com/tntwise/real-video-enhancer --branch 2.3.8

Cloning into 'real-video-enhancer'...
remote: Enumerating objects: 28926, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 28926 (delta 104), reused 103 (delta 90), pack-reused 28790 (from 3)
Receiving objects: 100% (28926/28926), 22.89 MiB | 37.62 MiB/s, done.
Resolving deltas: 100% (21309/21309), done.


In [ ]:
import os
!mkdir real-video-enhancer/models/
origdir = os.getcwd()
os.chdir("real-video-enhancer/models/")

def has_model(model: str):
    return model and model.lower() != "none"

def download_model(model: str):
    os.system(f"wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/{model}")



# get interpolate model by extracting the end of the link
def download_external_model(model_link : str):
        print("Downloading Model")
        os.system(f"wget {model_link}")
        model = model_link.split(r'/')[-1]
        return model


if INTERPOLATE:
    if "http" in INTERPOLATE_MODEL:
        UPSCALE_MODEL = download_external_model(INTERPOLATE_MODEL)
    else:
        download_model(INTERPOLATE_MODEL)
if UPSCALE:

    if "http" in UPSCALE_MODEL:
        print("up")
        UPSCALE_MODEL = download_external_model(UPSCALE_MODEL)
    else:
        download_model(UPSCALE_MODEL)


os.chdir(origdir)



In [ ]:
!mkdir backend # create backend dir so rve doesnt crash. This is to prevent rve from not detecting a "backend" folder, and being unable to write to a non existant log file location.

mkdir: cannot create directory ‘backend’: File exists


In [ ]:
if BACKEND.lower() == 'tensorrt':
    !wget https://github.com/astral-sh/python-build-standalone/releases/download/20250317/cpython-3.12.9+20250317-x86_64-unknown-linux-gnu-install_only.tar.gz
    !tar xvf cpython-3.12.9+20250317-x86_64-unknown-linux-gnu-install_only.tar.gz
    !./python/bin/python3 -m pip install  --extra-index-url https://download.pytorch.org/whl/cu126 -r real-video-enhancer/backend/requirements.txt # only install if using tensorrt

!mkdir bin/ && cd bin && wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/ffmpeg && chmod +x ffmpeg

mkdir: cannot create directory ‘bin/’: File exists


In [ ]:
#example upscale/denoise command
# It is recommended to use  --video_encoder_preset x264_nvenc
def build_backend_command() -> str:
    command = (('python3 real-video-enhancer/backend/rve-backend.py' if BACKEND == "pytorch" else './python/bin/python3 real-video-enhancer/backend/rve-backend.py')
               + f' -i "{INPUT_FILE}"'
               + f' -o "{OUTPUT_FILE}"'
               + f' -b {BACKEND} '
               + f' --video_encoder_preset {VIDEO_ENCODER}'
               + f' --audio_encoder_preset {AUDIO_ENCODER}'
               + f' --subtitle_encoder_preset {SUBTITLE_ENCODER}')
    if UPSCALE_MODEL and UPSCALE:
        command += f" --upscale_model real-video-enhancer/models/{UPSCALE_MODEL} "
    if INTERPOLATE_MODEL and INTERPOLATE:
        command += f" --interpolate_model real-video-enhancer/models/{INTERPOLATE_MODEL} "
        command += f" --interpolate_factor {INTERPOLATE_FACTOR} "
        command += f" --scene_detect_method {SCENE_DETECT_METHOD} "
        command += f" --scene_detect_threshold {SCENE_DETECT_SENSATIVITY} "

    return command

with open("command.sh", "w") as f:
  f.write(build_backend_command())

!bash command.sh

BACKEND: Getting Input Video Properties
Duration: 153.0 seconds
Total Frames: 765
Resolution: 820x616
FPS: 5.0
Color Space: None
Color Transfer: None
Color Primaries: None
Pixel Format: yuv420p
Video Codec: mpeg4
Video Bitrate: 933 kbps
Is HDR: False
Bit Depth: 8
RVE Backend Version: 2.3.8-dev13
CLI Arguments: 
['real-video-enhancer/backend/rve-backend.py', '-i', 'trimmed construction.mp4', '-o', 'output2.mkv', '-b', 'pytorch', '--video_encoder_preset', 'libx264', '--audio_encoder_preset', 'copy_audio', '--subtitle_encoder_preset', 'copy_subtitle', '--upscale_model', 'real-video-enhancer/models/2x_ModernSpanimationV2.pth', '--interpolate_model', 'real-video-enhancer/models/rife4.25.pkl', '--interpolate_factor', '2', '--scene_detect_method', 'pyscenedetect', '--scene_detect_threshold', '3.5']
BACKEND: Getting Input Video Properties
BACKEND: Using backend: pytorch
BACKEND: Setting up Upscale
BACKEND: No module named 'tensorrt'
BACKEND: No module named 'rife_ncnn_vulkan_python'
BACKEND: H